In [10]:
import optuna
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from optuna.samplers import TPESampler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBClassifier


In [2]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/train.csv", index_col='id')
test = pd.read_csv("/kaggle/input/tabular-playground-series-sep-2021/test.csv", index_col='id')

sample_submission = pd.read_csv('../input/tabular-playground-series-sep-2021/sample_solution.csv')

In [3]:
train.head()

In [4]:
features = [x for x in train.columns if x.startswith('f')]

In [5]:
train['n_missing'] = train[features].isna().sum(axis=1)
train['abs_sum'] = train[features].abs().sum(axis=1)
train['std'] = train[features].std(axis=1)
train['avg'] = train[features].mean(axis=1)
train['max'] = train[features].max(axis=1)
train['min'] = train[features].min(axis=1)

test['n_missing'] = test[features].isna().sum(axis=1)
test['abs_sum'] = test[features].abs().sum(axis=1)
test['std'] = test[features].std(axis=1)
test['avg'] = test[features].mean(axis=1)
test['max'] = test[features].min(axis=1)
test['min'] = test[features].min(axis=1)

In [6]:
%%time
train = train.apply(lambda x: x.fillna(x.median()),axis=0)
test = test.apply(lambda x: x.fillna(x.median()), axis=0)

In [7]:
X, X_val, y, y_val = train_test_split(train.drop('claim', axis=1), train.claim, test_size=0.2, random_state=42)

In [8]:
def create_model(trial):
    max_depth = trial.suggest_int("max_depth", 2, 6)
    n_estimators = trial.suggest_int("n_estimators", 1, 150)
    learning_rate = trial.suggest_uniform('learning_rate', 0.0000001, 1)
    num_leaves = trial.suggest_int("num_leaves", 2, 3000)
    min_child_samples = trial.suggest_int('min_child_samples', 3, 200)
    model = LGBMClassifier(
        learning_rate=learning_rate, 
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        num_leaves=num_leaves, 
        min_child_samples=min_child_samples,
        random_state=42
    )
    return model

class Optimizer:
    def __init__(self, metric, trials=15):
        self.metric = metric
        self.trials = trials
        self.sampler = TPESampler(seed=42)
        
    def objective(self, trial):
        model = create_model(trial)
        model.fit(X, y)
        preds = model.predict(X_val)
        if self.metric == 'acc':
            return accuracy_score(y_val, preds)
        else:
            return roc_auc_score(y_val, preds)
            
    def optimize(self):
        study = optuna.create_study(direction="maximize", sampler=self.sampler)
        study.optimize(self.objective, n_trials=self.trials)
        return study.best_params
    
optimizer = Optimizer('f1')
lgb_f1_params = optimizer.optimize()
lgb_f1_params['random_state'] = 42
lgb_f1 = LGBMClassifier(
    **lgb_f1_params
)
lgb_f1.fit(X, y)
preds = lgb_f1.predict(X_val)

print('Optimized on ROC AUC SCORE')
print('Optimized LightGBM accuracy: ', accuracy_score(y_val, preds))
print('Optimized LightGBM roc-auc-score: ', roc_auc_score(y_val, preds, average='macro'))

In [11]:
def create_model(trial):
    max_depth = trial.suggest_int("max_depth", 2, 6)
    n_estimators = trial.suggest_int("n_estimators", 1, 150)
    learning_rate = trial.suggest_uniform('learning_rate', 0.0000001, 1)
    gamma = trial.suggest_uniform('gamma', 0.0000001, 1)
    subsample = trial.suggest_uniform('subsample', 0.0001, 1.0)
    model = XGBClassifier(
        learning_rate=learning_rate, 
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        gamma=gamma, 
        subsample=subsample,
        random_state=42
    )
    return model

optimizer = Optimizer('f1')
xgb_f1_params = optimizer.optimize()
xgb_f1_params['random_state'] = 42
xgb_f1 = XGBClassifier(
    **xgb_f1_params
)
xgb_f1.fit(X, y)
preds = xgb_f1.predict(X_val)

print('Optimized on ROC AUC score')
print('Optimized XGBoost accuracy: ', accuracy_score(y_val, preds))
print('Optimized XGBoost roc-auc-score: ', roc_auc_score(y_val, preds))

In [ ]:
%%time
def create_model(trial):
    max_depth = trial.suggest_int("max_depth", 2, 6)
    n_estimators = trial.suggest_int("n_estimators", 2, 150)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 10)
    model = RandomForestClassifier(
        min_samples_leaf=min_samples_leaf, 
        n_estimators=n_estimators, 
        max_depth=max_depth, 
        random_state=42
    )
    return model

optimizer = Optimizer('roc_auc')
rf_f1_params = optimizer.optimize()
rf_f1_params['random_state'] = 42
rf_f1 = RandomForestClassifier(
    **rf_f1_params
)
rf_f1.fit(X, y)
preds = rf_f1.predict(X_val)

print('Optimized on ROC AUC score')
print('Optimized Random Forest: ', accuracy_score(y_val, preds))
print('Optimized Random Forest roc-auc-score: ', roc_auc_score(y_val, preds))

In [ ]:
from collections import Counter
from sklearn.tree import DecisionTreeClassifier
np.random.seed(42)
def bootstrap_sample(X, y):
    n_samples = X.shape[0]
    idxs = np.random.choice(n_samples, n_samples, replace=True)
    return X[idxs], y[idxs]


def most_common_label(y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common
class RandomForest:
    def __init__(self, n_trees=100):
        self.n_trees = n_trees

        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTreeClassifier(
                criterion='gini', max_depth=None, min_samples_split=2, 
                min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', max_leaf_nodes=None, 
                min_impurity_decrease=0.0, min_impurity_split=None,  
            )
            X_samp, y_samp = bootstrap_sample(X, y)
            tree.fit(X_samp, y_samp)
            self.trees.append(tree)

    def predict(self, X):
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        tree_preds = np.swapaxes(tree_preds, 0, 1)
        y_pred = [most_common_label(tree_pred) for tree_pred in tree_preds]
        return np.array(y_pred)

my_rf_cls = RandomForest()
my_rf_cls.fit(X_train.values, y_train.values)

In [ ]:
preds = my_rf_cls.predict(X_test)
print('Optimized LightGBM ROC AUC SCORE: ', roc_auc_score(y_test, preds))

In [ ]:
models = [
    ('lgbm', LGBMClassifier(**lgb_f1_params)),
    ('rf', RandomForestClassifier(**rf_f1_params)),
    ('xgboost', XGBClassifier(**xgb_f1_params))
]

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
oof_pred_tmp = dict()
test_pred_tmp = dict()
scores_tmp = dict()

for fold, (idx_train, idx_valid) in enumerate(kf.split(X, y)):
    X_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
    X_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
    
    for name, model in models:
        if name not in scores_tmp:
            oof_pred_tmp[name] = list()
            oof_pred_tmp['y_valid'] = list()
            test_pred_tmp[name] = list()
            scores_tmp[name] = list()
     
        model.fit(
            X_train, y_train,
            eval_set=[(X_valid,y_valid)],
            verbose=0
        )
        
        pred_valid = model.predict_proba(X_valid)[:,1]
        score = roc_auc_score(y_valid, pred_valid)
        
        scores_tmp[name].append(score)
        oof_pred_tmp[name].extend(pred_valid)
        
        print(f"Fold: {fold + 1} Model: {name} Score: {score}")
        print('--'*20)
        
        y_hat = model.predict_proba(test_data)[:,1]
        test_pred_tmp[name].append(y_hat)
    
    oof_pred_tmp['y_valid'].extend(y_valid)
        
for name, model in models:
    print(f"Overall Validation Score | {name}: {np.mean(scores_tmp[name])}")
    print('::'*20)